In [21]:
!pip install wordllama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 53.6 MB/s eta 0:00:00:00:0100:01


In [22]:
from wordllama import WordLlama

# Load the default WordLlama model
wl = WordLlama.load(trunc_dim=64)

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
video_data = pd.read_parquet('/kaggle/input/ohrenet-data-set-na-30-gb/video_stat.parquet')

In [3]:
video_data = video_data[~(video_data.title == "")]

In [4]:
drop_columns = ['v_is_hidden', 'v_is_deleted', 
                'v_frac_avg_watchtime_1_day_duration', 
                'v_frac_avg_watchtime_7_day_duration', 
                'v_frac_avg_watchtime_30_day_duration', 'row_number']
video_data.drop(columns = drop_columns, inplace=True)

In [5]:
video_data.reset_index(drop=True, inplace=True)

In [6]:
video_data['v_pub_datetime'] = pd.to_datetime(video_data['v_pub_datetime'])

In [16]:
video_data['v_pub_datetime'] = video_data['v_pub_datetime'].astype(str)

In [11]:
video_data['v_pub_datetime'] = video_data['v_pub_datetime'].apply(lambda x: x.timestamp())

In [17]:
video_data_dict = video_data[['video_id', 'v_pub_datetime', 'title', 'description', 'category_id']].set_index('video_id').T.to_dict('list')

In [19]:
import pickle 

with open('video_data_dict.pkl', 'wb') as f:
    pickle.dump(video_data_dict, f)

In [13]:
((video_data.groupby('category_id').count()[['video_id']]/len(video_data)) * 100).sort_values('video_id')

,video_id
category_id,
Дизайн,0.174059
Аудио,0.187048
Недвижимость,0.191356
Природа,0.282790
Наука,0.397629
Психология,0.401809
Охота и рыбалка,0.414669
Аудиокниги,0.445211
Бизнес и предпринимательство,0.473889


In [14]:
video_data.isna().sum()

video_id                                 0
v_pub_datetime                           0
v_total_comments                         0
v_year_views                             0
v_month_views                            0
v_week_views                             0
v_day_views                              0
v_likes                                  0
v_dislikes                               0
v_duration                               0
v_cr_click_like_7_days                   0
v_cr_click_dislike_7_days                0
v_cr_click_vtop_7_days                   0
v_cr_click_long_view_7_days              0
v_cr_click_comment_7_days                0
v_cr_click_like_30_days                  0
v_cr_click_dislike_30_days               0
v_cr_click_vtop_30_days                  0
v_cr_click_long_view_30_days             0
v_cr_click_comment_30_days               0
v_cr_click_like_1_days                   0
v_cr_click_dislike_1_days                0
v_cr_click_vtop_1_days                   0
v_cr_click_

In [17]:
category_id_embeds = wl.embed(list(video_data['category_id'].values))

In [18]:
title_embeds = wl.embed(list(video_data['title'].values))

In [19]:
description_embeds = wl.embed(list(video_data['description'].values))

In [26]:
video_embeds = category_id_embeds + title_embeds*0.5 + description_embeds*0.5

In [27]:
video_embeds

array([[-0.5831768 ,  0.28563842,  0.275738  , ...,  0.17058557,
        -0.06529325, -0.338973  ],
       [-0.664851  ,  0.7072224 ,  0.00805275, ..., -0.17414491,
        -0.05017198, -0.23635134],
       [-0.79016507,  0.31092542, -0.65970737, ..., -0.14822435,
        -0.0459419 , -1.1075921 ],
       ...,
       [-0.52784103,  0.7532077 ,  0.30392748, ..., -0.59711385,
         0.20593771, -0.7534518 ],
       [ 0.03709494,  0.36053303, -0.32807246, ..., -0.07837309,
         0.36607343, -0.52025425],
       [-0.23801419,  0.5977721 , -0.27375567, ..., -0.13478799,
         0.26058307, -0.5754348 ]], dtype=float32)

In [28]:
stats_vecs = video_data.iloc[:, 1:-4].values

In [29]:
stats_vecs

array([[1.71848148e+09, 0.00000000e+00, 5.56000000e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.61875606e+09, 0.00000000e+00, 1.09000000e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.66869220e+09, 0.00000000e+00, 1.63000000e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.66771590e+09, 0.00000000e+00, 1.62000000e+02, ...,
        0.00000000e+00, 0.00000000e+00, 9.00000000e+00],
       [1.70017413e+09, 0.00000000e+00, 1.44000000e+02, ...,
        0.00000000e+00, 2.00000000e+00, 8.00000000e+00],
       [1.70024604e+09, 0.00000000e+00, 2.51000000e+03, ...,
        0.00000000e+00, 4.90000000e+01, 1.53000000e+02]])

In [30]:
video_embeds = np.concatenate((video_embeds, stats_vecs), axis=1)

In [31]:
video_embeds

array([[-5.83176792e-01,  2.85638422e-01,  2.75738001e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-6.64851010e-01,  7.07222402e-01,  8.05275328e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-7.90165067e-01,  3.10925424e-01, -6.59707367e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-5.27841032e-01,  7.53207684e-01,  3.03927481e-01, ...,
         0.00000000e+00,  0.00000000e+00,  9.00000000e+00],
       [ 3.70949432e-02,  3.60533029e-01, -3.28072459e-01, ...,
         0.00000000e+00,  2.00000000e+00,  8.00000000e+00],
       [-2.38014191e-01,  5.97772121e-01, -2.73755670e-01, ...,
         0.00000000e+00,  4.90000000e+01,  1.53000000e+02]])

72 столбец это v_duration

In [32]:
pd.DataFrame(video_embeds)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
0,-0.583177,0.285638,0.275738,-0.121871,0.564524,-0.365153,0.362858,-0.751143,0.385155,0.538049,0.645825,-0.004905,0.729265,0.495231,0.119218,0.053123,-0.218398,0.742045,-0.290449,0.172615,0.067378,-0.789981,0.600600,-0.104020,-0.323671,0.085661,0.100123,0.507746,0.179764,0.051270,0.834238,-0.228952,-0.083433,-0.128188,-0.076265,0.494338,-0.223548,0.072786,1.092361,-0.305275,0.371989,0.396749,0.160727,0.511020,0.292119,0.159505,0.541094,-0.394530,-0.532157,0.078351,1.168953,-0.061255,-0.081888,-0.309979,-0.649225,0.175444,-0.546548,0.529301,0.263943,-0.906742,-0.467546,0.170586,-0.065293,-0.338973,1.718481e+09,0.0,556.0,2.0,0.0,0.0,0.0,0.0,7.733,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.656386,1.616750,0.0,0.0,0.0
1,-0.664851,0.707222,0.008053,-0.751596,0.373044,-0.562850,-0.102521,-0.893505,0.395862,-0.162864,0.521615,-0.220767,-0.031076,0.137717,-0.925609,-0.214496,0.180183,0.943128,0.022646,-0.183280,-0.269995,-0.034704,0.854077,0.251240,-0.297983,0.152702,0.608045,-0.354415,-0.322021,0.149512,0.612997,-0.777970,-0.044872,-0.510096,-0.437391,1.015516,-0.093101,0.117605,0.936895,0.435847,0.227154,0.028993,-0.363164,-0.724855,0.373487,0.465612,-0.149527,0.300928,-0.158997,-0.690141,0.575692,0.126717,-0.171996,-0.248849,-0.797535,0.816011,0.455228,0.201020,0.010713,-0.254389,-0.708024,-0.174145,-0.050172,-0.236351,1.618756e+09,0.0,109.0,2.0,1.0,0.0,0.0,0.0,379.000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,-1.000000,40.500000,5.570802,5.649635,0.0,0.0,0.0
2,-0.790165,0.310925,-0.659707,-0.643019,0.406513,0.042416,0.208823,-0.862269,-0.328970,0.692598,-0.065213,-0.709210,0.059631,0.507024,0.871917,-0.364344,0.104277,1.018446,-0.230907,-1.150446,-0.007015,-0.109687,0.543142,0.675645,0.784227,0.594423,0.333407,-1.104720,-0.625381,0.269102,1.229691,0.339700,-0.479583,-0.244641,-0.186320,1.854517,0.123798,0.309722,1.150039,0.328661,0.634249,0.019243,-0.049112,-0.560453,-0.363031,-0.834832,0.298770,-0.078194,0.257752,-0.719543,0.165512,0.886866,0.518570,0.185060,-0.593253,0.218857,-0.000081,-0.309548,0.444951,-0.813440,0.368732,-0.148224,-0.045942,-1.107592,1.668692e+09,0.0,163.0,2.0,2.0,0.0,0.0,0.0,1020.767,0.0,0.0,50.0,0.000000,0.0,0.00000,0.0,50.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,160.000000,160.000000,0.342614,0.285310,0.0,0.0,0.0
3,-1.801291,0.815261,-0.024182,-0.223238,0.202299,-0.440562,0.153128,-0.455030,0.294274,0.594064,0.357509,-0.709632,-0.038351,0.562807,-0.166207,-0.168631,-0.060880,1.022655,0.124486,-0.716810,-0.548097,0.014735,-0.042354,0.181423,-0.103773,1.087313,-0.090668,0.269261,-0.680937,-0.030069,1.041744,-0.815783,0.404358,-0.403535,-0.475185,0.536770,-0.072338,0.731002,1.419756,-0.310014,0.684217,0.423880,-0.097656,0.169474,0.211458,-0.312884,-0.033532,0.607763,-0.983823,-0.173747,2.133065,0.287571,0.384681,0.111129,-0.342804,0.581238,-0.158161,1.296779,-0.250414,-0.788803,-0.335349,-0.335656,-0.006601,-0.889414,1.262293e+09,0.0,10657.0,5227.0,1300.0,202.0,24.0,0.0,42.025,0.0,0.0,0.0,89.384615,0.0,0.40176,0.0,0.0,87.048020,0.0,0.0,0.0,0.0,92.079208,0.0,57.579208,97.190000,122.290606,0.418336,0.513562,186.0,1162.0,4550.0
4,-0.051967,0.315467,-0.382886,-0.416392,0.510738,-0.441189,-0.089106,-0.616665,-0.140649,-0.422775,0.052300,-0.011423,0.376656,-0.021345,-0.877051,-0.444594,-0.489612,1.101015,-0.389657,0.127566,-0.720832,0.400770,0.709844,-0.386637,-0.131038,0.207414,0.838347,-1.059825,-0.029094,-0.463696,0.870700,-0.861045,-0.276225,-0.731685,-0.148527,1.044264,0.061544,-0.512945,0.507302,0.734434,0.186210,-0.275806,-0.653719,-0.529479,-0.030314,0.503932,0.098804,-0.555060,-0.070373,-0.562403,0.942571,0.270362,0

In [33]:
result_video_embeds = pd.concat([video_data['video_id'],pd.DataFrame(video_embeds)], axis=1)

In [34]:
result_video_embeds.to_orc('result_video_embeds_all.orc')